In [1]:
import numpy as np
from scipy.stats import uniform
from datetime import datetime
from datetime import timedelta

from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, \
                                               ConstantVelocity
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState

np.random.seed(1991)

start_time = datetime.now().replace(microsecond=0)
transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(0.005),
                                                          ConstantVelocity(0.005)])
timesteps = [start_time]
truth = GroundTruthPath([GroundTruthState([0, 1, 0, 1], timestamp=timesteps[0])])

for k in range(1, 21):
    timesteps.append(start_time+timedelta(seconds=k))
    truth.append(GroundTruthState(
        transition_model.function(truth[k-1], noise=True, time_interval=timedelta(seconds=1)),
        timestamp=timesteps[k]))



In [2]:
uniform.rvs(0,20)

15.606704711533384

In [3]:
prob_det = 0.9

In [4]:
from stonesoup.types.detection import TrueDetection
from stonesoup.types.detection import Clutter
from stonesoup.models.measurement.linear import LinearGaussian

measurement_model = LinearGaussian(
    ndim_state = 4,
    mapping = (0,2),
    noise_covar = np.array([[0.75, 0], 
                           [0, 0.75]])
)

all_measurements = []

for state in truth:
    measurement_set = set()

    # Generate actual detection from the state with a 1-pd chance that no detection is received
    if np.random.rand() <= prob_det:
        measurement = measurement_model.function(state, noise = True)
        measurement_set.add(TrueDetection(state_vector = measurement,
                                          groundtruth_path = truth,
                                          timestamp = state.timestamp,
                                          measurement_model = measurement_model))
    # Generate clutter at this time-step
    truth_x = state.state_vector[0]
    truth_y = state.state_vector[2]
    for _ in range(np.random.randint(10)):
        x = uniform.rvs(truth_x-10, 20)
        y = uniform.rvs(truth_y-10, 20)
        measurement_set.add(Clutter(np.array([[x],[y]]),
                                    timestamp = state.timestamp,
                                    measurement_model = measurement_model))
    all_measurements.append(measurement_set)


In [5]:
from stonesoup.plotter import AnimatedPlotterly

plotter = AnimatedPlotterly(timesteps, tail_length=1)
plotter.plot_ground_truths(truth, [0,2])
plotter.plot_measurements(all_measurements, [0,2])

plotter.fig

In [6]:
from stonesoup.predictor.kalman import KalmanPredictor
predictor = KalmanPredictor(transition_model)
from stonesoup.updater.kalman import KalmanUpdater
updater = KalmanUpdater(measurement_model)

from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.measures import Mahalanobis
hypothesiser = DistanceHypothesiser(predictor, updater, measure=Mahalanobis(), missed_distance=3)

In [7]:
from stonesoup.dataassociator.neighbour import NearestNeighbour
data_associator = NearestNeighbour(hypothesiser)

In [8]:
# Create prior
from stonesoup.types.state import GaussianState
prior = GaussianState([[0], [1], [0], [1]], np.diag([1.5, 0.5, 1.5, 0.5]), timestamp=start_time)

In [15]:
from stonesoup.types.track import Track

track = Track([prior])
for n, measurements in enumerate(all_measurements):
    hypotheses = data_associator.associate([track],
                                           measurements,
                                           start_time + timedelta(seconds=n))
    
    hypothesis = hypotheses[track]
    
    if hypothesis.measurement:
        post = updater.update(hypothesis)
        track.append(post)
    else:  # When data associator says no detections are good enough, we'll keep the prediction
        track.append(hypothesis.prediction)

In [16]:
plotter.plot_tracks(track, [0, 2], uncertainty=True)
plotter.fig